In [11]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [13]:
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder \
    .appName("Home Sales Analysis") \
    .getOrCreate()

25/01/12 02:25:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [15]:
import os
file_path = "/Users/latifahjones/Desktop/Home_Sales/Resources/home_sales_revised.csv"
print(os.path.exists(file_path))  # This should return True if the file exists

True


In [17]:
# Check if the file exists
if os.path.exists(file_path):
    try:
        # Read the CSV file into a DataFrame
        df = spark.read.csv(file_path, header=True, inferSchema=True)
        
        # Create a temporary view of the DataFrame
        df.createOrReplaceTempView("home_sales")
        
        # Display the DataFrame to confirm it loaded correctly
        df.show()
    except Exception as e:
        print(f"Error reading the CSV file: {e}")
else:
    print(f"The file does not exist at the specified path: {file_path}")

+--------------------+--------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|    date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+--------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|  4/8/22|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...| 6/13/21|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...| 4/12/19|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|10/16/19|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|  1/8/22|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...| 1/30/19|      2017|218712|       2|        3|       1965|   14375|     2|

In [89]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
average_price_query = """
SELECT
    YEAR(date) AS year,
    ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY YEAR(date)
ORDER BY year
"""

average_price_df = spark.sql(average_price_query)
average_price_df.show()

+----+-------------+
|year|average_price|
+----+-------------+
+----+-------------+



In [25]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
average_price_built_query = """
SELECT 
    date_built AS year_built, 
    ROUND(AVG(price), 2) AS average_price 
FROM 
    home_sales 
WHERE 
    bedrooms = 3 AND bathrooms = 3 
GROUP BY 
    date_built 
ORDER BY 
    year_built
"""

# Run the query and create a DataFrame
average_price_built_df = spark.sql(average_price_built_query)

# Show the results
average_price_built_df.show()

+----------+-------------+
|year_built|average_price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [27]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
average_price_built_query = """
SELECT 
    date_built AS year_built, 
    ROUND(AVG(price), 2) AS average_price 
FROM 
    home_sales 
WHERE 
    bedrooms = 3 
    AND bathrooms = 3 
    AND floors = 2 
    AND sqft_living >= 2000 
GROUP BY 
    date_built 
ORDER BY 
    year_built
"""

# Run the query and create a DataFrame
average_price_built_df = spark.sql(average_price_built_query)

# Show the results
average_price_built_df.show()

+----------+-------------+
|year_built|average_price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [29]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.

# Start the timer
start_time = time.time()

# SQL query to calculate the average price of homes per view rating
average_price_view_query = """
SELECT 
    view, 
    ROUND(AVG(price), 2) AS average_price 
FROM 
    home_sales 
WHERE 
    price >= 350000 
GROUP BY 
    view 
ORDER BY 
    view DESC
"""

# Run the query and create a DataFrame
average_price_view_df = spark.sql(average_price_view_query)

# Show the results
average_price_view_df.show()

# Calculate and print the runtime
print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.9010839462280273 seconds ---


In [31]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

DataFrame[]

In [33]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [35]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

average_price_view_query = """
SELECT 
    view, 
    ROUND(AVG(price), 2) AS average_price 
FROM 
    home_sales 
WHERE 
    price >= 350000 
GROUP BY 
    view 
ORDER BY 
    view DESC
"""

# Start the timer for the cached query
start_time = time.time()

# Run the query using the cached data and create a DataFrame
average_price_view_cached_df = spark.sql(average_price_view_query)

# Show the results
average_price_view_cached_df.show()

# Calculate and print the runtime for the cached query
print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.6685879230499268 seconds ---


In [101]:
# 11. Read the formatted parquet data.
# Define the output path for the partitioned Parquet data
output_path = "/Users/latifahjones/Desktop/Home_Sales/Resources/home_sales_partitioned"

# Read the formatted Parquet data back into a DataFrame
df_parquet = spark.read.parquet(output_path)

# Show the DataFrame to verify the data
df_parquet.show()

+--------------------+--------------------+-----+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|                date|price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+--------------------+-----+--------+---------+-----------+--------+------+----------+----+----------+
|�E 4X@0-99ab-7a...|                NULL| NULL|    NULL|     NULL|       NULL|    NULL|  NULL|      NULL|NULL|      NULL|
|�Ob5e-730c7dd...|                NULL| NULL|    NULL|     NULL|       NULL|    NULL|  NULL|      NULL|NULL|      NULL|
|�\t &)�\fm�...|                NULL| NULL|    NULL|     NULL|       NULL|    NULL|  NULL|      NULL|NULL|      NULL|
|  �\bTP9cc�!�1...|                NULL| NULL|    NULL|     NULL|       NULL|    NULL|  NULL|      NULL|NULL|      NULL|
|L DItiAJ !v 8Ai...|                NULL| NULL|    NULL|     NULL|       NULL|    NULL|  NULL|      NULL|NULL|      NULL|
|P\;�

In [103]:
output_path = "/Users/latifahjones/Desktop/Home_Sales/Resources/"
df_parquet = spark.read.parquet(output_path)
df_parquet.show()

Py4JJavaError: An error occurred while calling o213.parquet.
: java.lang.AssertionError: assertion failed: Conflicting directory structures detected. Suspicious paths:
	file:/users/latifahjones/desktop/home_sales/resources/home_sales_partitioned
	file:/users/latifahjones/desktop/home_sales/resources/home_sales_revised.csv
	file:/users/latifahjones/desktop/home_sales/resources/parquet_home_sales

If provided paths are partition directories, please set "basePath" in the options of the data source to specify the root directory of the table. If there are multiple root directories, please load them separately and then union them.
	at scala.Predef$.assert(Predef.scala:223)
	at org.apache.spark.sql.execution.datasources.PartitioningUtils$.parsePartitions(PartitioningUtils.scala:178)
	at org.apache.spark.sql.execution.datasources.PartitioningUtils$.parsePartitions(PartitioningUtils.scala:110)
	at org.apache.spark.sql.execution.datasources.PartitioningAwareFileIndex.inferPartitioning(Partition

In [105]:
# 12. Create a temporary table for the parquet data.


In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 14. Uncache the home_sales temporary table.


In [ ]:
# 15. Check if the home_sales is no longer cached

